# Testing Stopwords and Stemmers

## Portuguese Stopwords

In [1]:
from nltk.corpus import stopwords

stopwords_list = stopwords.words('portuguese')

# Remover algumas palavras da lista, p.ex. "não" para manter a negação
stopwords_list.remove('não')

print(stopwords_list)

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos',

In [2]:
text = "Esta unidade curricular de Processamento de Linguagem Natural é extremamente divertida e útil porque nos ajuda a desenvolver novas competências no campo da Aprendizagem Computacional"

In [3]:
' '.join([w for w in text.split() if w not in stopwords_list])

'Esta unidade curricular Processamento Linguagem Natural extremamente divertida útil porque ajuda desenvolver novas competências campo Aprendizagem Computacional'

## Trying some Portuguese Stemmers

In [4]:
from nltk.stem import PorterStemmer, RSLPStemmer
from nltk.stem.snowball import PortugueseStemmer

ps = PorterStemmer()
snb = PortugueseStemmer()
rslp = RSLPStemmer()

text_list = text.split()

print("Porter Stemmer:\n", ' '.join([ps.stem(w) for w in text_list]))
print("\nSnowball Portugue Stemmer:\n", ' '.join([snb.stem(w) for w in text_list]))
print("\nRSLP Stemmer:\n",' '.join([rslp.stem(w) for w in text_list]))

Porter Stemmer:
 esta unidad curricular de processamento de linguagem natur é extremament divertida e útil porqu no ajuda a desenvolv nova competência no campo da aprendizagem computacion

Snowball Portugue Stemmer:
 esta unidad curricul de process de linguag natural é extrem divert e útil porqu nos ajud a desenvolv nov competent no camp da aprendizag computacional

RSLP Stemmer:
 est unidad curricul de process de lingu natur é extrem divert e útil porqu no ajud a desenvolv nov compet no camp da aprendiz computac


#### Dataset

In [5]:
import pandas as pd

dataset = pd.read_excel('OpArticles_ADUs.xlsx')

Base code to test the different Stemmers with a TfidfVectorizer and ComplementNB Classifier, later compare their metrics

In [6]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import confusion_matrix, classification_report
import time

stopwords_list = stopwords.words('portuguese')
stopwords_list.remove('não')

vectorizer = TfidfVectorizer()

y = dataset['label']

def test_stemmer(stemmer):
    corpus = []

    # Preprocess
    start = time.time()
    for i in range(0, dataset['tokens'].size):
        # get review, remove and lowercase non alpha chars
        review = re.sub('[^a-zA-Z\u00C0-\u00ff]', ' ', dataset['tokens'][i]).lower()
        # split into tokens, apply stemming and remove stop words
        review = ' '.join([stemmer.stem(w) for w in review.split() if w not in stopwords_list])
        corpus.append(review)
    stop = time.time()
    print("Preprocess time: %0.2fs" % (stop - start))

    print(corpus[:5])

    # Fit vectorizer
    X = vectorizer.fit_transform(corpus).toarray()
    print("\n(Number of samples, Number of features):", X.shape)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0, stratify=y)

    start = time.time()
    clf = ComplementNB()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    stop = time.time()

    print("\nModel time: %0.2fs" % (stop - start))
    print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("\nClassification report:\n", classification_report(y_test, y_pred))

### Porter

In [7]:
test_stemmer(ps)

Preprocess time: 4.05s
['facto não apena fruto ignorância', 'havia humor jornalismo investigação preocupação aprofundar contextualizar história isenção relato preocupação social urgência denunciar muita peça realment jornalística', 'tudo cómico fifa', 'todo permitimo organização faça totalment absurdo sentido', 'não fazem rir custa poderoso']

(Number of samples, Number of features): (16743, 15170)

Model time: 1.22s

Confusion matrix:
 [[306  22 277  44  84]
 [  4  57  59   8   5]
 [278  71 966  99 207]
 [ 43  22 105  93  19]
 [ 70  33 187  20 270]]

Classification report:
               precision    recall  f1-score   support

        Fact       0.44      0.42      0.43       733
      Policy       0.28      0.43      0.34       133
       Value       0.61      0.60      0.60      1621
    Value(+)       0.35      0.33      0.34       282
    Value(-)       0.46      0.47      0.46       580

    accuracy                           0.51      3349
   macro avg       0.43      0.45     

### Portuguese Snowball

In [8]:
test_stemmer(snb)

Preprocess time: 4.87s
['fact nã apen frut ignor', 'hav humor jornal investig preocup aprofund contextualiz histór isençã relat preocup social urgênc denunc muit pec realment jornalíst', 'tud cómic fif', 'tod permit organiz fac total absurd sent', 'nã faz rir cust poder']

(Number of samples, Number of features): (16743, 9148)

Model time: 0.62s

Confusion matrix:
 [[304  29 287  42  71]
 [  9  38  71   8   7]
 [260  73 981 108 199]
 [ 40  12 117  90  23]
 [ 66  25 189  20 280]]

Classification report:
               precision    recall  f1-score   support

        Fact       0.45      0.41      0.43       733
      Policy       0.21      0.29      0.25       133
       Value       0.60      0.61      0.60      1621
    Value(+)       0.34      0.32      0.33       282
    Value(-)       0.48      0.48      0.48       580

    accuracy                           0.51      3349
   macro avg       0.42      0.42      0.42      3349
weighted avg       0.51      0.51      0.51      3349



### RSLP

In [9]:
test_stemmer(rslp)

Preprocess time: 7.28s
['fact não apen frut ignor', 'hav hum jorn investig preocup aprofund contextual histór isenç relat preocup soc urg denunci muit peç real jorn', 'tud cómic fif', 'tod permit organiz faç total absurd sent', 'não faz rir cust poder']

(Number of samples, Number of features): (16743, 8256)

Model time: 0.39s

Confusion matrix:
 [[300  28 288  42  75]
 [  6  41  70   9   7]
 [263  66 968 112 212]
 [ 42  11 115  93  21]
 [ 69  28 196  18 269]]

Classification report:
               precision    recall  f1-score   support

        Fact       0.44      0.41      0.42       733
      Policy       0.24      0.31      0.27       133
       Value       0.59      0.60      0.59      1621
    Value(+)       0.34      0.33      0.33       282
    Value(-)       0.46      0.46      0.46       580

    accuracy                           0.50      3349
   macro avg       0.41      0.42      0.42      3349
weighted avg       0.50      0.50      0.50      3349



### Conclusions

The _Porter Stemmer_  had similar metrics to the others, but it does not make sense to use it since it is not a Stemmer aimed to Portuguese text, was only used as a matter of comparison.

We will use the _RSLP Stemmer_ as it trims more the word endings and leads to less features, causing smaller matrixes for the classifiers and thus faster times and less resources.
Even though the preprocess time was double compared to the _Snowball Portuguese Stemmer_, the metrics are simliar, and it is only run once in the preprocess before the classification tasks, so the advantages outweight this disavantage.